In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-11-09T17:26:09.897323+00:00!"


In [2]:
import pandas as pd
import os 
import sqlalchemy
import time

In [3]:
with open("/home/kali/COVID-19-ES/airflow/process/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/kali/COVID-19-ES/airflow/download_file/downloads/Informe_Comunicacion_20211108.ods


In [4]:
#Here we load the excel file download for the official web site the health ministry
#Remove the 3 last lines Total, vaccins of Armed forces and Health professionals
#Leaving the sheet_name empty refers will take the first sheet in the excel file
dataframe = pd.read_excel(input_file)
dataframe

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Dosis entregadas Janssen (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Nº Personas con dosis adicional,Fecha de la última vacuna registrada (2)
0,Andalucía,9884250,2019500,1528900,333500,13766150,13229714,0.961032,6855610,6752567,374751,2021-11-07 00:00:00
1,Aragón,1517367,267720,264060,63815,2112962,2035376,0.963281,1068093,1049564,45521,2021-11-05 00:00:00
2,Asturias,1280885,162000,220900,43000,1706785,1719509,1.007455,874057,866122,53998,2021-11-07 00:00:00
3,Baleares,1318950,305100,256400,57600,1938050,1615210,0.833420,859888,843635,7421,2021-11-07 00:00:00
4,Canarias,2403150,574700,455800,133650,3567300,3187906,0.893647,1681910,1642890,36297,2021-11-07 00:00:00
5,Cantabria,747195,114900,125200,22200,1009495,940314,0.931470,483096,477623,24367,2021-11-06 00:00:00
6,Castilla y Leon,3274545,411830,473110,101025,4260510,3802260,0.892442,2004593,1965878,62850,2021-11-03 00:00:00
7,Castilla La Mancha,2330505,446800,396400,95075,3268780,3080248,0.942323,1621034,1563970,68854,2021-11-06 00:00:00
8,Cataluña,8368460,1604900,1514300,375750,11863410,11345956,0.956382,5989564,5866345,215457,2021-11-07 00:00:00
9,C. Valenciana,5797540,1213650,863900,213450,8088540,7831632,0.968238,4080223,4015726,195222,2021-11-07 00:00:00


In [5]:
cols_names_dict = {'Unnamed: 0': 'community',
             'Total Dosis entregadas (1)': 'Total doses delivered',
             'Dosis administradas (2)': 'Doses administered',
             '% sobre entregadas': '% over delivered',
             'Nº Personas vacunadas(pauta completada)': 'No. People vaccinated',
             'Fecha de la última vacuna registrada (2)': 'Date'}

In [6]:
#rename columns
dataframe.rename(columns=cols_names_dict, inplace=True)
dataframe = dataframe[list(cols_names_dict.values())]
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,Date
0,Andalucía,13766150,13229714,0.961032,6752567,2021-11-07 00:00:00
1,Aragón,2112962,2035376,0.963281,1049564,2021-11-05 00:00:00
2,Asturias,1706785,1719509,1.007455,866122,2021-11-07 00:00:00
3,Baleares,1938050,1615210,0.833420,843635,2021-11-07 00:00:00
4,Canarias,3567300,3187906,0.893647,1642890,2021-11-07 00:00:00
5,Cantabria,1009495,940314,0.931470,477623,2021-11-06 00:00:00
6,Castilla y Leon,4260510,3802260,0.892442,1965878,2021-11-03 00:00:00
7,Castilla La Mancha,3268780,3080248,0.942323,1563970,2021-11-06 00:00:00
8,Cataluña,11863410,11345956,0.956382,5866345,2021-11-07 00:00:00
9,C. Valenciana,8088540,7831632,0.968238,4015726,2021-11-07 00:00:00


In [7]:
communities = ['Andalucía',
         'Aragón',
         'Asturias ',
         'Baleares',
         'C. Valenciana',
         'Canarias',
         'Cantabria',
         'Castilla La Mancha',
         'Castilla y Leon ',
         'Cataluña',
         'Ceuta',
         'Extremadura',
         'Galicia',
         'La Rioja',
         'Madrid',
         'Melilla',
         'Murcia ',
         'Navarra',
         'País Vasco'
]
dataframe = dataframe.loc[dataframe["community"].isin(communities)]
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,Date
0,Andalucía,13766150,13229714,0.961032,6752567,2021-11-07 00:00:00
1,Aragón,2112962,2035376,0.963281,1049564,2021-11-05 00:00:00
2,Asturias,1706785,1719509,1.007455,866122,2021-11-07 00:00:00
3,Baleares,1938050,1615210,0.833420,843635,2021-11-07 00:00:00
4,Canarias,3567300,3187906,0.893647,1642890,2021-11-07 00:00:00
5,Cantabria,1009495,940314,0.931470,477623,2021-11-06 00:00:00
6,Castilla y Leon,4260510,3802260,0.892442,1965878,2021-11-03 00:00:00
7,Castilla La Mancha,3268780,3080248,0.942323,1563970,2021-11-06 00:00:00
8,Cataluña,11863410,11345956,0.956382,5866345,2021-11-07 00:00:00
9,C. Valenciana,8088540,7831632,0.968238,4015726,2021-11-07 00:00:00


In [8]:
#Add ccaa_iso columns (iso code of communities)
def format_community(community_name) : 
    communities = {
        "Andalucía": "AN" ,
        "Aragón": "AR",
        "Asturias": "AS",
        "Canarias": "CN",
        "Cantabria": "CB",
        "Castilla y Leon": "CL",
        "Castilla La Mancha": "CM",
        "Cataluña": "CT",
        "Extremadura": "EX",
        "Galicia": "GA",
        "Baleares": "IB",
        "La Rioja": "RI",
        "Madrid": "MD",
        "Murcia": "MC",
        "Navarra": "NC",
        "País Vasco" : "PV",
        "C. Valenciana": "VC",
        "Ceuta": "CE", 
        "Melilla" : "ML"
    }
    return communities[community_name.strip()] if community_name.strip() in list(communities.keys()) else community_name

dataframe["ccaa_iso"] = dataframe["community"].apply(format_community)
#Set index to date
dataframe = dataframe.set_index("Date")
dataframe

/home/kali/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,ccaa_iso
Date,,,,,,
2021-11-07,Andalucía,13766150,13229714,0.961032,6752567,AN
2021-11-05,Aragón,2112962,2035376,0.963281,1049564,AR
2021-11-07,Asturias,1706785,1719509,1.007455,866122,AS
2021-11-07,Baleares,1938050,1615210,0.833420,843635,IB
2021-11-07,Canarias,3567300,3187906,0.893647,1642890,CN
2021-11-06,Cantabria,1009495,940314,0.931470,477623,CB
2021-11-03,Castilla y Leon,4260510,3802260,0.892442,1965878,CL
2021-11-06,Castilla La Mancha,3268780,3080248,0.942323,1563970,CM
2021-11-07,Cataluña,11863410,11345956,0.956382,5866345,CT


In [9]:
def connection(attempts = 3):
    try :
        database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://u5487210@cvp19:jRG2XXi4CwjHvAa@cvp19.mysql.database.azure.com/my_db')
        return database_connection
    except : 
        if attempts>0:
            print("Error!")
            time.sleep(30)
            connection(attempts-1)
    return False


In [10]:
if connection() : 
    dataframe.to_sql(con=connection(), name='vaccins', if_exists='append')
else : 
    print("The error persists")

In [11]:
os.system(f'mv "{input_file}" "/home/kali/COVID-19-ES/airflow/process/processed/"')

0